In [1]:
import sys
import regex as re
import numpy as np
import io
import matplotlib.pyplot as plt
import pandas as pd
import math
import h5py
import time
import os
import datetime
import pickle

LOCAL = True
BASE_DIR = '../'

if not LOCAL:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_DIR = '/content/drive/My Drive/Colab Notebooks/thesis/'

sys.path.append(BASE_DIR + 'lib')
sys.path.append(BASE_DIR + 'config')
from preprocessing import process_sakt, save_h5, transpose_list, process_one_feature, select_from_rows
import dataset_parameters as params

# DATASET = 'akribian'
# DATASET = 'assistments_2009'
# DATASET = 'assistments_2012'
# DATASET = 'junyi_academy'
DATASET = 'ednet'

INPUT_DIR = BASE_DIR + 'data/' + DATASET + '/raw/'
OUTPUT_DIR = BASE_DIR + 'data/' + DATASET + '/processed/sakt/'
FILE_NAME = 'sorted.csv'

TIME_STEPS = params.time_steps_dict[DATASET]
VALIDATION_RATIO = 0.2
TEST_RATIO = 0.05
SPLIT_SECTIONS = 5

ROWS_PER_READ = 1000000
SHUFFLE = False

STRIDE = params.stride_dict[DATASET]
CALCULATE_NUM_EXERCISES = True

## Load the data

In [2]:
# Columns to read from dataset
columns = params.columns_dict[DATASET]
group_column = columns[0]
exercise_columns = columns[1]
exercise_id_columns = columns[3]
encoding = params.encodings_dict[DATASET]

data = pd.read_csv(INPUT_DIR + "rawdata.csv", encoding = encoding)
data['idx'] = data.index
data.sort_values([group_column, 'idx'], inplace=True)

In [3]:
data.drop(columns = ['idx'])
data.to_csv(INPUT_DIR + FILE_NAME)

In [4]:
data

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,idx
0,0,0,115,5692,0,1,3,1,NaN,NaN,0
1,1,56943,115,5716,0,2,2,1,37000.0,False,1
2,2,118363,115,128,0,0,0,1,55000.0,False,2
3,3,131167,115,7860,0,3,0,1,19000.0,False,3
4,4,137965,115,7922,0,4,1,1,11000.0,False,4
...,...,...,...,...,...,...,...,...,...,...,...
101230327,101230327,428564420,2147482888,3586,0,22,0,1,18000.0,True,101230327
101230328,101230328,428585000,2147482888,6341,0,23,3,1,14000.0,True,101230328
101230329,101230329,428613475,2147482888,4212,0,24,3,1,14000.0,True,101230329
101230330,101230330,428649406,2147482888,6343,0,25,1,0,22000.0,True,101230330
